In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
from src.dataset import *
from src.concept_vectors import *
from src.util import *
from src.plots import *
from src.hierarchy import *
from src.metrics import *
from src.models import * 
from src.create_vectors import *

2023-03-28 13:46:45.785305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image, ImageOps
import pickle
import sklearn
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from scipy.spatial.distance import cdist


In [5]:
dataset = CUB_Dataset()

In [6]:
model_name = "VGG16"

## Exploration of the Truthfulness Metric

In [7]:
similarities = get_model_concept_similarities(dataset,model)

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 196s 5s/step


In [8]:
attributes = dataset.get_attributes()

In [11]:
shapley_hierarchy = create_hierarchy(create_ward_hierarchy, load_shapley_vectors_simple,dataset,'',attributes,43)

In [12]:
print(shapley_hierarchy)

                ---- has_belly_color::brown
           ----|
                     ---- has_underparts_color::brown
                ----|
                     ---- has_breast_color::brown


      ----|
                     ---- has_head_pattern::eyebrow
                ----|
                     ---- has_wing_pattern::spotted

           ----|
                               ---- has_back_pattern::striped
                          ----|
                               ---- has_wing_pattern::striped

                     ----|
                               ---- has_throat_color::buff
                          ----|
                                         ---- has_upper_tail_color::buff
                                    ----|
                                         ---- has_under_tail_color::buff

                               ----|
                                         ---- has_nape_color::buff
                                    ----|
                                             

In [20]:
attributes = np.array(attributes)

In [23]:
for i in range(5):
    rand_index = random.randint(0,111)
    five_largest = np.argsort(similarities[rand_index])[::-1][:5]
    print(attributes[rand_index],attributes[five_largest])

has_wing_color::yellow ['has_wing_color::yellow' 'has_upperparts_color::yellow'
 'has_forehead_color::yellow' 'has_back_pattern::multi-colored'
 'has_crown_color::yellow']
has_wing_color::black ['has_wing_color::black' 'has_back_color::black'
 'has_upperparts_color::black' 'has_primary_color::black'
 'has_under_tail_color::black']
has_bill_length::about_the_same_as_head ['has_bill_length::about_the_same_as_head' 'has_size::medium_(9_-_16_in)'
 'has_bill_shape::dagger' 'has_primary_color::white'
 'has_bill_shape::hooked_seabird']
has_wing_pattern::striped ['has_wing_pattern::striped' 'has_back_pattern::striped'
 'has_upperparts_color::buff' 'has_under_tail_color::buff'
 'has_wing_color::buff']
has_size::small_(5_-_9_in) ['has_size::small_(5_-_9_in)' 'has_bill_length::shorter_than_head'
 'has_shape::perching-like' 'has_tail_shape::notched_tail'
 'has_bill_shape::cone']


In [28]:
truthfulness_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43])

2023-03-20 10:56:47.812288: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


(-0.24, 0.0)

## Re-train large models

In [147]:
train_large_image_model(dataset,model_name,"_image_robustness",seed=43)

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Seed 43 suffix _image_robustness
Found 4796 validated image filenames belonging to 200 classes.
Generated dataset, now training model
Epoch 1/25
150/150 [==============================] - 54s 355ms/step - loss: 5.7406 - accuracy: 0.0052
Epoch 2/25
150/150 [==============================] - 32s 215ms/step - loss: 5.0942 - accuracy: 0.0152
Epoch 3/25
150/150 [==============================] - 32s 213ms/step - loss: 4.7786 - accuracy: 0.0288
Epoch 4/25
150/150 [==============================] - 33s 220ms/step - loss: 4.4631 - accuracy: 0.0548
Epoch 5/25
150/150 [==============================] - 33s 218ms/step - loss: 4.1591 - accuracy: 0.0867
Epoch 6/25
150/150 [==============================] - 32s 215ms/step - loss: 3.8844 - accuracy: 0.1188
Epoch 7/25
150/150 [==============================] - 36s 237ms/step - loss: 3.5902 - accuracy: 0.1695
Epoch 8/25
150/150 [==============================] - 36s 242ms/step - loss: 3.3094 - accuracy: 0.2252
Epoch 9/25
150/150 [======================

In [148]:
create_shapley_vectors(dataset.get_attributes(),dataset,"_image_robustness",seed=43)

Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 34s 226ms/step


In [149]:
robustness_image_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43])

(0.35119047619047616, 0.0)

## Create Shapley Vectors

In [19]:
create_shapley_vectors(dataset.get_attributes(),dataset,"",seed=42)

Found 4796 validated image filenames belonging to 200 classes.


In [24]:
truthfulness_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[42])

2023-03-21 11:40:48.241735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10802 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:82:00.0, compute capability: 6.1


(0.04, 0.0)

In [86]:
create_shapley_vectors(dataset.get_attributes(),dataset,"",seed=43)

Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 34s 226ms/step


In [87]:
create_shapley_vectors(dataset.get_attributes(),dataset,"",seed=44)

Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 35s 229ms/step


In [88]:
create_shapley_vectors(dataset.get_attributes(),dataset,"",seed=45)

Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 34s 225ms/step


In [62]:
create_shapley_vectors(dataset.get_attributes(),dataset,"_image_robustness",seed=43)

Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 36s 237ms/step


In [63]:
create_shapley_vectors(dataset.get_attributes(),dataset,"_image_responsiveness",seed=43)

Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 35s 233ms/step


In [33]:
for embedding_method in [load_label_vectors_simple,load_vae_concept_vectors_simple,load_cem_vectors_simple,load_concept2vec_vectors_simple,load_cem_stratified_vectors_simple]:
    embedding_method(dataset.get_attributes()[0],dataset,"",seed=43).shape

In [51]:
create_shapley_vectors(dataset.get_attributes(),dataset,"",seed=45)

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 197s 5s/step


In [99]:
create_shapley_vectors(MNIST_Dataset().get_attributes(),MNIST_Dataset(),"",seed=43)

Found 60000 validated image filenames belonging to 10 classes.
1875/1875 [==============================] - 159s 85ms/step


## Compute Truthfulness Scores

In [38]:
for embedding_method in [load_shapley_vectors_simple]:
    print(embedding_method)
    t = truthfulness_metric_shapley(embedding_method,dataset,dataset.get_attributes(),[43,44,45],model_name="VGG16")
    print("{} {}".format(embedding_method,t))

<function load_shapley_vectors_simple at 0x7fbf55bae9e0>
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 197s 5s/step
<function load_shapley_vectors_simple at 0x7fbf55bae9e0> (0.18666666666666665, 0.3714236873915765)


In [58]:
truthfulness_metric_shapley(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[42],model_name="VGG16")

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 226ms/step
Computed all concept embeddings


(0.8017857142857144, 0.0)

In [90]:
truthfulness_metric_shapley(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43],model_name="VGG16")

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 223ms/step
Computed all concept embeddings


(0.7535714285714287, 0.0)

In [91]:
truthfulness_metric_shapley(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[44],model_name="VGG16")

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 232ms/step
Computed all concept embeddings


(0.7535714285714287, 0.0)

In [92]:
truthfulness_metric_shapley(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[45],model_name="VGG16")

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 223ms/step
Computed all concept embeddings


(0.7535714285714287, 0.0)

In [84]:
truthfulness_metric_shapley(load_label_vectors_simple,dataset,dataset.get_attributes(),[43],model_name="VGG16")

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 235ms/step
Computed all concept embeddings


(0.7303571428571428, 0.0)

In [111]:
truthfulness_metric_shapley(load_random_vectors_simple,dataset,dataset.get_attributes(),[42],model_name="VGG16")

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 227ms/step
Computed all concept embeddings


(0.033928571428571426, 0.0)

## Evaluating Shapley Vectors across metrics

In [96]:
truthfulness_metric_shapley(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43,44,45])

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 233ms/step
Computed all concept embeddings
Computed all concept embeddings
Computed all concept embeddings


(0.7535714285714287, 0.0)

In [97]:
for embedding_method in [load_label_vectors_simple,load_cem_vectors_simple,load_vae_vectors_simple,load_concept2vec_vectors_simple]:
    t = truthfulness_metric_shapley(embedding_method,dataset,dataset.get_attributes(),[43,44,45])
    print(embedding_method,t)

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 223ms/step
Computed all concept embeddings
Computed all concept embeddings
Computed all concept embeddings
<function load_label_vectors_simple at 0x7f7388b77d40> (0.7303571428571428, 0.0)
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 228ms/step


/auto/homes/njr61/main_code/src/metrics.py:349: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_concept_embeddings = np.array([embedding_method(i,dataset,"",seed=seed) for i in dataset.get_attributes()])


Computed all concept embeddings
Computed all concept embeddings
Computed all concept embeddings
<function load_cem_vectors_simple at 0x7f7388acb050> (0.05476190476190477, 0.0030351306628528447)
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 229ms/step
Computed all concept embeddings
Computed all concept embeddings
Computed all concept embeddings
<function load_vae_vectors_simple at 0x7f7388acbc20> (0.20773809523809525, 0.05051464168875505)
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 234ms/step
Computed all concept embeddings
Computed all concept embeddings
Computed all concept embeddings
<function load_concept2vec_vectors_simple at 0x7f7388acba70> (0.41130952380952385, 0.012400396819047418)


In [93]:
stability_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43,44,45])

(0.005952380952380931, 0.002430049347999176)

In [94]:
robustness_image_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43])

(0.33630952380952384, 0.0)

In [95]:
responsiveness_image_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43])

(0.3839285714285714, 0.0)

## Exploration of Shapley Vectors <-> Truthfulness relationship

In [53]:
model_name = "VGG16"

In [54]:
model = get_large_image_model(dataset,model_name)
model.load_weights("results/models/{}_models/{}_45.h5".format(model_name.lower(),dataset.experiment_name))

similarity_matrix = get_model_concept_similarities(dataset,model)

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 197s 5s/step


In [55]:
attributes = dataset.get_attributes()

In [56]:
seed = 45

In [83]:
all_embeddings = [embedding_method(i,dataset,"",seed=44)[0] for i in dataset.get_attributes()]
all_embeddings = np.array(all_embeddings)
distances = cdist(all_embeddings,all_embeddings,metric='cosine')

In [84]:
attribute = "has_eye_color::black"
attribute_idx = dataset.get_attributes().index(attribute)

In [85]:
co_occuring_concepts = find_similar_conepts_shapley(attribute,dataset,similarity_matrix,5)

In [86]:
top_5_concepts = np.argsort(distances[attribute_idx])[1:1+5]
co_occuring_concepts_hierarchy = np.array(dataset.get_attributes())[top_5_concepts]

In [87]:
co_occuring_concepts

['has_back_pattern::solid',
 'has_shape::perching-like',
 'has_breast_pattern::solid',
 'has_bill_shape::all-purpose',
 'has_throat_color::yellow']

In [88]:
co_occuring_concepts_hierarchy

array(['has_tail_shape::notched_tail', 'has_size::small_(5_-_9_in)',
       'has_back_pattern::solid', 'has_shape::perching-like',
       'has_bill_length::shorter_than_head'], dtype='<U39')

In [76]:
selected_concepts = random.sample(attributes,k=50)
selected_concepts_indices = [attributes.index(i) for i in selected_concepts]

temp_truthfulness = []

for concept in selected_concepts:
    co_occuring_concepts = find_similar_conepts_shapley(concept,dataset,similarity_matrix,5)
    co_occuring_concepts_hierarchy = rank_distance_concepts(embedding_method,dataset,concept,
                                                            co_occuring_concepts,seed)

    if len(co_occuring_concepts) == 1:
        temp_truthfulness.append(int(co_occuring_concepts == co_occuring_concepts_hierarchy))
    else:
        score = stats.kendalltau(co_occuring_concepts,
                                                  co_occuring_concepts_hierarchy).correlation
        
        if score < 0:
            print(concept,score)
            print(co_occuring_concepts)
            print(co_occuring_concepts_hierarchy)
        
        temp_truthfulness.append(score)
np.mean(temp_truthfulness)

has_forehead_color::white -0.19999999999999998
['has_crown_color::white', 'has_nape_color::white', 'has_back_color::white', 'has_primary_color::white', 'has_under_tail_color::white']
['has_crown_color::white', 'has_nape_color::white', 'has_under_tail_color::white', 'has_back_color::white', 'has_primary_color::white']
has_bill_shape::cone -0.19999999999999998
['has_back_pattern::striped', 'has_tail_shape::notched_tail', 'has_wing_pattern::striped', 'has_upperparts_color::buff', 'has_bill_length::shorter_than_head']
['has_tail_shape::notched_tail', 'has_bill_length::shorter_than_head', 'has_back_pattern::striped', 'has_wing_pattern::striped', 'has_upperparts_color::buff']
has_under_tail_color::buff -0.19999999999999998
['has_upper_tail_color::buff', 'has_upperparts_color::buff', 'has_throat_color::buff', 'has_wing_pattern::striped', 'has_wing_color::brown']
['has_upper_tail_color::buff', 'has_upperparts_color::buff', 'has_wing_color::brown', 'has_throat_color::buff', 'has_wing_pattern::s

0.3559999999999999

KeyboardInterrupt: 

In [43]:
robustness_image_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43])

(0.7738095238095238, 0.0)

In [44]:
responsiveness_image_metric(load_shapley_vectors_simple,dataset,dataset.get_attributes(),[43])

(0.36309523809523814, 0.0)

In [100]:
shapley_hierarchy_mnist = create_hierarchy(create_ward_hierarchy, load_shapley_vectors_simple,MNIST_Dataset(),'',MNIST_Dataset().get_attributes(),43)

In [101]:
print(shapley_hierarchy_mnist)

           ---- 0_color
      ----|
           ---- 0_number

 ----|
                ---- 1_color
           ----|
                ---- 1_number

      ----|
                     ---- 2_color
                ----|
                     ---- 2_number

           ----|
                          ---- 7_color
                     ----|
                          ---- 7_number

                ----|
                               ---- 9_color
                          ----|
                               ---- 9_number

                     ----|
                                    ---- 8_color
                               ----|
                                    ---- 8_number

                          ----|
                                         ---- 6_color
                                    ----|
                                         ---- 6_number

                               ----|
                                              ---- 3_color
                                      

In [102]:
shapley_hierarchy_cub_44 = create_hierarchy(create_ward_hierarchy, load_shapley_vectors_simple,CUB_Dataset(),'',CUB_Dataset().get_attributes(),44)

In [103]:
shapley_hierarchy_cub_45 = create_hierarchy(create_ward_hierarchy, load_shapley_vectors_simple,CUB_Dataset(),'',CUB_Dataset().get_attributes(),45)

In [104]:
print(shapley_hierarchy_cub_44)

                     ---- has_breast_color::buff
                ----|
                          ---- has_underparts_color::buff
                     ----|
                          ---- has_belly_color::buff


           ----|
                     ---- has_throat_color::buff
                ----|
                               ---- has_upper_tail_color::buff
                          ----|
                               ---- has_under_tail_color::buff

                     ----|
                                    ---- has_upperparts_color::buff
                               ----|
                                         ---- has_wing_color::buff
                                    ----|
                                         ---- has_back_color::buff


                          ----|
                                    ---- has_nape_color::buff
                               ----|
                                    ---- has_primary_color::buff





      ----|
                   

In [105]:
print(shapley_hierarchy_cub_45)

                          ---- has_upper_tail_color::buff
                     ----|
                          ---- has_under_tail_color::buff

                ----|
                               ---- has_upperparts_color::buff
                          ----|
                                    ---- has_wing_color::buff
                               ----|
                                    ---- has_back_color::buff


                     ----|
                               ---- has_nape_color::buff
                          ----|
                               ---- has_primary_color::buff



           ----|
                     ---- has_throat_color::buff
                ----|
                          ---- has_breast_color::buff
                     ----|
                               ---- has_underparts_color::buff
                          ----|
                               ---- has_belly_color::buff




      ----|
                     ---- has_belly_color::brown
          

In [114]:
shapley_robustness_hierarchy_cub_43 = create_hierarchy(create_ward_hierarchy, load_label_vectors_simple,CUB_Dataset(),'_image_robustness',CUB_Dataset().get_attributes(),43)

In [115]:
print(shapley_robustness_hierarchy_cub_43)

                     ---- has_throat_color::white
                ----|
                          ---- has_breast_color::white
                     ----|
                               ---- has_underparts_color::white
                          ----|
                               ---- has_belly_color::white



           ----|
                               ---- has_upper_tail_color::grey
                          ----|
                               ---- has_under_tail_color::grey

                     ----|
                               ---- has_wing_color::grey
                          ----|
                                    ---- has_primary_color::grey
                               ----|
                                         ---- has_upperparts_color::grey
                                    ----|
                                         ---- has_back_color::grey




                ----|
                          ---- has_bill_length::about_the_same_as_head
               

In [113]:
label_hierarchy_cub_43 = create_hierarchy(create_ward_hierarchy,load_shapley_vectors_simple,CUB_Dataset(),'',CUB_Dataset().get_attributes(),43)

In [108]:
print(label_hierarchy_cub_43)

                     ---- has_throat_color::white
                ----|
                          ---- has_breast_color::white
                     ----|
                               ---- has_underparts_color::white
                          ----|
                               ---- has_belly_color::white



           ----|
                               ---- has_upper_tail_color::grey
                          ----|
                               ---- has_under_tail_color::grey

                     ----|
                               ---- has_wing_color::grey
                          ----|
                                    ---- has_primary_color::grey
                               ----|
                                         ---- has_upperparts_color::grey
                                    ----|
                                         ---- has_back_color::grey




                ----|
                          ---- has_bill_length::about_the_same_as_head
               

In [129]:
normal_vector_distances = flat_distance_to_square(get_concept_distances(load_shapley_vectors_simple,dataset,"",dataset.get_attributes(),43))
normal_vector_distances_2 = flat_distance_to_square(get_concept_distances(load_shapley_vectors_simple,dataset,"",dataset.get_attributes(),44))
robust_vector_distances = flat_distance_to_square(get_concept_distances(load_shapley_vectors_simple,dataset,"_image_robustness",dataset.get_attributes(),43))

In [135]:
k = 5

pairs_1 = get_top_k_pairs(normal_vector_distances,k=k)
pairs_2 = get_top_k_pairs(robust_vector_distances,k=k)
pairs_3 = get_top_k_pairs(normal_vector_distances_2,k=k)

intersection = set(pairs_1).intersection(set(pairs_2))
len(intersection)/(len(normal_vector_distances)*k)

0.725

## Export Concept Vectors

In [98]:
for seed in [43,44,45]:
    for name, embedding_method in zip(["labels","shapley","cem","vae","concept2vec"],[load_label_vectors_simple,load_shapley_vectors_simple,
                                                            load_cem_vectors_simple,load_vae_vectors_simple,load_concept2vec_vectors_simple]):
        all_embeddings =  np.array([embedding_method(i,CUB_Dataset(),"",seed=seed)[0] for i in CUB_Dataset().get_attributes()])
        np.save(open("results/temp/{}_{}.npy".format(name,seed),"wb"),all_embeddings)

In [30]:
model = get_large_image_model(dataset,model_name)
model.load_weights("results/models/{}_models/{}_42.h5".format(model_name.lower(),dataset.experiment_name))

AttributeError: 'Functional' object has no attribute 'eval'

In [53]:
similarity_matrix = get_model_concept_similarities(dataset,model)
similarity_matrix.shape

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 229ms/step


(112, 200)

In [54]:
shapley_vectors = np.array([load_shapley_vectors_simple(i,dataset,"",seed=42)[0] for i in dataset.get_attributes()])

In [55]:
shapley_vectors

array([[-0.00154002, -0.00228546, -0.00373522, ..., -0.00281988,
        -0.00386812,  0.00771958],
       [ 0.03377431,  0.01300337, -0.00574072, ..., -0.00516757,
        -0.00387471,  0.00981994],
       [-0.00426624, -0.00549528,  0.00836984, ...,  0.00303919,
         0.00029663, -0.00529514],
       ...,
       [-0.00088717, -0.00367799, -0.0030079 , ..., -0.0044956 ,
        -0.00101499, -0.00109161],
       [-0.00324722, -0.00442993, -0.00368103, ...,  0.00027132,
         0.00821149, -0.00416407],
       [ 0.00096188, -0.00145153, -0.0026877 , ...,  0.00486752,
        -0.00419491, -0.0029747 ]], dtype=float32)

In [56]:
similarity_matrix

array([[-0.00154002, -0.00228546, -0.00373522, ..., -0.00281988,
        -0.00386812,  0.00771958],
       [ 0.03377431,  0.01300337, -0.00574072, ..., -0.00516757,
        -0.00387471,  0.00981994],
       [-0.00426624, -0.00549528,  0.00836985, ...,  0.00303919,
         0.00029663, -0.00529514],
       ...,
       [-0.00088717, -0.00367799, -0.0030079 , ..., -0.0044956 ,
        -0.00101499, -0.00109161],
       [-0.00324722, -0.00442993, -0.00368103, ...,  0.00027132,
         0.00821149, -0.00416407],
       [ 0.00096188, -0.00145153, -0.0026877 , ...,  0.00486752,
        -0.00419491, -0.0029747 ]], dtype=float32)

In [27]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32
image_size = (224, 224)

num_classes = len(set([i['class_label'] for i in dataset.get_data()]))
num_attributes = len(dataset.get_data()[0]['attribute_label'])

data_valid = dataset.get_data(train=False)
img_paths_valid = ['dataset/'+i['img_path'] for i in data_valid]
labels_valid = [str(i['class_label']) for i in data_valid]
valid_df = pd.DataFrame(zip(img_paths_valid,labels_valid), columns=["image_path", "label"])

valid_generator = datagen.flow_from_dataframe(dataframe=valid_df,
                                          x_col="image_path",
                                          y_col="label",
                                          target_size=image_size,
                                          batch_size=batch_size,
                                          class_mode="categorical",
                                          shuffle=False)


Found 1198 validated image filenames belonging to 200 classes.


In [31]:
predictions = model.predict(valid_generator)

38/38 [==============================] - 8s 213ms/step


In [47]:
concepts = np.array([i['attribute_label'] for i in data_valid])
contribution_array = np.array([[contribution_score(concepts,predictions,concept_num,class_num) for class_num in range(num_classes)]  for concept_num in range(num_attributes)])

In [48]:
contribution_array

array([[0, 0, 0, ..., 0, 0, 1],
       [3, 0, 0, ..., 0, 0, 1],
       [0, 0, 4, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 0, 0]])

In [46]:
contribution_score(concepts,predictions,1,0)

[ True False False ... False False False]
74
1198


3

In [49]:
data = dataset.get_data(train=False)
img_paths = ['dataset/'+i['img_path'] for i in data]
labels = [str(i['class_label']) for i in data]

num_attributes = len(dataset.get_attributes())
num_classes = len(set(labels))

concept_vectors = {}
model_name = "VGG16"
model = get_large_image_model(dataset,model_name)
model.load_weights("results/models/{}_models/{}_{}.h5".format(model_name.lower(),dataset.experiment_name+'',42))

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32
image_size = (224, 224)

valid_df = pd.DataFrame(zip(img_paths,labels), columns=["image_path", "label"])

valid_generator = datagen.flow_from_dataframe(dataframe=valid_df,
                                          x_col="image_path",
                                          y_col="label",
                                          target_size=image_size,
                                          batch_size=batch_size,
                                          class_mode="categorical",
                                          shuffle=False)

predictions = model.predict(valid_generator)

concepts = np.array([i['attribute_label'] for i in data[:len(predictions)]])
contribution_array = np.array([[contribution_score(concepts,predictions,concept_num,class_num) 
                                for class_num in range(num_classes)] for concept_num in range(num_attributes)])


Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 9s 228ms/step


In [50]:
contribution_array

array([[-0.00154002, -0.00228546, -0.00373522, ..., -0.00281988,
        -0.00386812,  0.00771958],
       [ 0.03377431,  0.01300337, -0.00574072, ..., -0.00516757,
        -0.00387471,  0.00981994],
       [-0.00426624, -0.00549527,  0.00836985, ...,  0.00303919,
         0.00029663, -0.00529514],
       ...,
       [-0.00088717, -0.00367799, -0.0030079 , ..., -0.0044956 ,
        -0.00101499, -0.00109161],
       [-0.00324722, -0.00442992, -0.00368103, ...,  0.00027132,
         0.0082115 , -0.00416407],
       [ 0.00096188, -0.00145153, -0.0026877 , ...,  0.00486752,
        -0.00419491, -0.0029747 ]], dtype=float32)

In [76]:
similarity_matrix = get_model_concept_similarities(dataset,model)

Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 8s 215ms/step


In [77]:
similarity_matrix

array([[ 1.        ,  0.17645355, -0.35855821, ...,  0.23737512,
        -0.28935193,  0.00461934],
       [ 0.17645355,  1.        , -0.4931272 , ..., -0.0991226 ,
        -0.35281454, -0.0486328 ],
       [-0.35855821, -0.4931272 ,  1.        , ..., -0.16656851,
         0.13145783,  0.10951849],
       ...,
       [ 0.23737512, -0.0991226 , -0.16656851, ...,  1.        ,
         0.03719031, -0.29870522],
       [-0.28935193, -0.35281454,  0.13145783, ...,  0.03719031,
         1.        , -0.44677931],
       [ 0.00461934, -0.0486328 ,  0.10951849, ..., -0.29870522,
        -0.44677931,  1.        ]])

In [70]:
concept = dataset.get_attributes()[0]
compare_concepts = 5
all_concept_embeddings = np.array([
    load_shapley_vectors_simple(i,dataset,"",seed=42) for i in dataset.get_attributes()])

In [71]:
all_concept_embeddings.shape

(112, 1, 200)

In [74]:
co_occuring_concepts = find_similar_conepts_shapley(concept,dataset,similarity_matrix,compare_concepts)
co_occuring_concepts_hierarchy = rank_distance_concepts(all_concept_embeddings,dataset,concept,
                                                        dataset.get_attributes(),42)[1:1+compare_concepts]

In [75]:
co_occuring_concepts

['has_bill_shape::hooked_seabird',
 'has_bill_shape::all-purpose',
 'has_bill_shape::cone',
 'has_wing_color::brown',
 'has_wing_color::grey']

## Create model

In [117]:
num_attributes = len([i['attribute_label'] for i in dataset.get_data()][0])
if model_name.lower() == "vgg16":
    model_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
else:
    raise Exception("{} model not implemented yet".format(model_name))

for layer in model_base.layers:
    layer.trainable = False

fc1 = Flatten()(model_base.output)
fc1 = Dense(1024, activation='relu')(fc1)
output = Dense(num_attributes, activation='sigmoid')(fc1)

model = Model(inputs=model_base.input, outputs=output)

model.compile(loss='binary_crossentropy',
          optimizer=Adam(lr=0.0001),
          metrics=['accuracy'])

In [118]:
data = dataset.get_data()

In [119]:
img_paths = ['dataset/'+i['img_path'] for i in data]
attributes = [i['attribute_label'] for i in data]

data_dict = {'image_path': img_paths}
for i in range(num_attributes):
    data_dict['attribute' + str(i)] = [int(attribute[i]) for attribute in attributes]

df = pd.DataFrame(data_dict)

# Create an ImageDataGenerator with the desired data augmentation settings
datagen = ImageDataGenerator(
    rescale=1./255,)

batch_size = 32
image_size = (224, 224)

data = dataset.get_data(seed=43,suffix='')
generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col='image_path',
    y_col=['attribute' + str(i) for i in range(num_attributes)],
    batch_size=batch_size,
    target_size=image_size,
    class_mode='other')


Found 4796 validated image filenames.


In [120]:
model.fit(generator,
       steps_per_epoch=len(generator),
       epochs=10)

Epoch 1/10
150/150 [==============================] - 34s 221ms/step - loss: 0.4119 - accuracy: 0.0102
Epoch 2/10
122/150 [=======================>......] - ETA: 6s - loss: 0.3545 - accuracy: 0.0087

KeyboardInterrupt: 

In [7]:
data = dataset.get_data()

In [8]:
img_paths = ['dataset/'+i['img_path'] for i in data]
attributes = np.array([i['attribute_label'] for i in data])
num_attributes = len(attributes[0])

In [9]:
# Define a function to load and preprocess the images
def load_image(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=input_shape[:2])
    img = tf.keras.utils.img_to_array(img) / 255.
    return img


In [10]:
input_shape = (224, 224, 3)


In [11]:
images = np.array([load_image(img_path) for img_path in img_paths])

In [12]:
input_tensor = Input(shape=input_shape)

# Define the base model
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add the new layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
# Define the output layer
output_layer = Dense(num_attributes, activation='sigmoid')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=['mse'])


2023-03-28 13:52:31.568657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 13:52:38.834458: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9699 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:03:00.0, compute capability: 6.1
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
epochs = 10
batch_size = 32
model.fit(images, attributes, batch_size=batch_size, epochs=epochs)

2023-03-28 13:52:41.736412: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2887729152 exceeds 10% of free system memory.
2023-03-28 13:52:45.297048: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2887729152 exceeds 10% of free system memory.


Epoch 1/10


2023-03-28 13:52:49.638878: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-03-28 13:52:51.278114: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


150/150 [==============================] - 18s 74ms/step - loss: 0.4188 - mse: 0.1299
Epoch 2/10
150/150 [==============================] - 9s 62ms/step - loss: 0.3293 - mse: 0.1025
Epoch 3/10
150/150 [==============================] - 10s 64ms/step - loss: 0.2767 - mse: 0.0853
Epoch 4/10
150/150 [==============================] - 10s 63ms/step - loss: 0.2264 - mse: 0.0686
Epoch 5/10
150/150 [==============================] - 10s 65ms/step - loss: 0.1867 - mse: 0.0556
Epoch 6/10
150/150 [==============================] - 10s 67ms/step - loss: 0.1524 - mse: 0.0444
Epoch 7/10
150/150 [==============================] - 10s 65ms/step - loss: 0.1221 - mse: 0.0344
Epoch 8/10
150/150 [==============================] - 10s 66ms/step - loss: 0.0968 - mse: 0.0263
Epoch 9/10
150/150 [==============================] - 10s 64ms/step - loss: 0.0768 - mse: 0.0199
Epoch 10/10
150/150 [==============================] - 10s 65ms/step - loss: 0.0607 - mse: 0.0149


In [28]:
model(np.array([images[0]])) > 0.5

<tf.Tensor: shape=(1, 112), dtype=bool, numpy=
array([[False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False,  True, False, False, False,
        False, False,  True, False,  True, False, False, False, False,
        False, False,  True, False, False, False, False, False,  True,
        False, False,  True, False, False, False, False,  True, False,
        False, False, False,  True, False,  True,  True, False, False,
        False, False, False, False,  True, False, False, False,  True,
        False, False, False, False, False,  True, False, False, False,
        False, False,  True, False, False, False, False,  True, False,
        False, False,  True, False, False,  True, False, False,  True,
        False, False, False, False,  True, False, False,  True, False,
        False, False, False, False, False, False, False, False,  True,
         True, False, False, False]])>

In [29]:
attributes[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0])

In [14]:
model.save_weights("results/models/vgg16_models/cub_concept.h5")

In [16]:
model(images,batch_size=32)

2023-03-28 13:56:14.156415: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2887729152 exceeds 10% of free system memory.


TypeError: call() got an unexpected keyword argument 'batch_size'

In [17]:
logits = model.predict(images, batch_size=32)

2023-03-28 14:00:51.779153: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2887729152 exceeds 10% of free system memory.
2023-03-28 14:01:04.584419: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.69GiB (rounded to 2887729152)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-03-28 14:01:04.584517: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-03-28 14:01:04.584589: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 45, Chunks in use: 44. 11.2KiB allocated for chunks. 11.0KiB in use in bin. 716B client-requested in use in bin.
2023-03-28 14:01:04.584608: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 6,

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.